<a href="https://colab.research.google.com/github/anparashar/mU-Net/blob/main/mU_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## Loading Dataset
#from simple_unet_model import simple_unet_model   #Use normal unet model
from tensorflow.keras.utils import normalize
#mport normalize
import os
import cv2
from PIL import Image
import numpy as np



image_directory = 'image directory'
mask_directory = 'mask directory'



SIZEX = 256

SIZEY= 256

image_dataset = []
mask_dataset = []

#Loading images
images = os.listdir(image_directory)
for i, image_name in enumerate(images):    #Remember enumerate method adds a counter and returns the enumerate object
    if (image_name.split('.')[1] == 'jpeg'):
        image = cv2.imread(image_directory+image_name,1)
        image = Image.fromarray(image)
        image = image.resize((SIZEX, SIZEY))
        image_dataset.append(np.array(image))

## Loading Mask

masks = os.listdir(mask_directory)
for i, image_name in enumerate(masks):
    if (image_name.split('.')[1] == 'jpeg'):
        mask = cv2.imread(mask_directory+image_name,0)
        mask = Image.fromarray(mask)
        mask = mask.resize((SIZEX, SIZEY))
        mask_dataset.append(np.array(mask))


        #Normalize images
image_dataset = normalize(np.array(image_dataset), axis=0)

mask_dataset = (np.array(mask_dataset))/255




In [ ]:
# train test split
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(image_dataset, mask_dataset, test_size = 0.20, random_state = 0)


In [ ]:
# creating train, test and validation set
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = 0.50, random_state = 0)

print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
# Sanity check
import random
import numpy as np
image_number = random.randint(0, len(X_train))
print(image_number)
plt.imshow(X_train[image_number])
plt.show()
plt.subplot(122)
plt.imshow(y_train[image_number], cmap='gray')
plt.show()

print(image_dataset.shape)
print(mask_dataset.shape)
NofImage =image_dataset.shape[0]
IMG_HEIGHT =image_dataset.shape[1]
IMG_WIDTH  = image_dataset.shape[2]
IMG_CHANNELS =image_dataset.shape[3]






In [ ]:
## mU-Net Model

#!pip install segmentation-models
from keras.models import Model
from keras.layers import Input, Conv2D,AveragePooling2D, MaxPooling2D,GlobalAveragePooling2D, UpSampling2D,BatchNormalization, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Nadam, Adagrad, Adadelta
from keras import backend as K
import tensorflow as tf
import numpy as np
#import segmentation_models as sm
from keras.callbacks import LearningRateScheduler
################################################################


def mUnet_model(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS):
#Build the model

    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    #s = Lambda(lambda x: x / 255)(inputs)   #No need for this if we normalize our inputs beforehand
    s = inputs

    #Contraction path
    c1 = Conv2D(75, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
    c1 = Dropout(0.2)(c1)
    c1 = Conv2D(75, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)

    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(150, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = Dropout(0.2)(c2)
    c2 = Conv2D(150, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)

    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(300, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(300, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)

    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(600, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(600, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)

    p4 = MaxPooling2D(pool_size=(2, 2))(c4)

    c5 = Conv2D(1200, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = Dropout(0.2)(c5)
    c5 = Conv2D(1200, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)


    #Expansive path
    u6 = Conv2DTranspose(600, (2, 2), strides=(2, 2), padding='same')(c5)
    m1= MaxPooling2D((8, 8))(c1)
    l1= concatenate([u6, m1])
    #k1= Conv2D(480, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    u6 = concatenate([l1, c4])
    c6 = Conv2D(600, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(600, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)


    u7 = Conv2DTranspose(300, (2, 2), strides=(2, 2), padding='same')(c6)
    m2= MaxPooling2D((4, 4))(c1)
    l2= concatenate([u7, m2])
    #k2= Conv2D(240, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    u7 = concatenate([l2, c3])
    c7 = Conv2D(300, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(300, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)

    u8 = Conv2DTranspose(150, (2, 2), strides=(2, 2), padding='same')(c7)
    m3= MaxPooling2D((2, 2))(c1)
    l3= concatenate([u8, m3])
    #k3= Conv2D(120, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    u8 = concatenate([l3, c2])
    c8 = Conv2D(150, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = Dropout(0.2)(c8)
    c8 = Conv2D(150, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)


    u9 = Conv2DTranspose(75, (2, 2), strides=(2, 2), padding='same')(c8)
   # m4= MaxPooling2D((2, 2))(c1)
    l4= concatenate([u9, c1])
    #k4= Conv2D(60, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    u9 = concatenate([l4, c1], axis=3)
    c9 = Conv2D(75, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = Dropout(0.2)(c9)
    c9 = Conv2D(75, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)


    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer= 'Adam' , loss= 'categorical_crossentropy')
    model.summary()

    return model

In [ ]:
from tensorflow.keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda

# calling model function
def get_model():
    return mUnet_model(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)
model = get_model()

# Training of model

def lr_decay(epoch):
        initAlpha=0.001
        factor=0.5
        dropEvery=7
        alpha=initAlpha*(factor ** np.floor((1+epoch)/dropEvery))
        return float(alpha)

callbacks=[LearningRateScheduler(lr_decay)]

history = model.fit(X_train, y_train,
                    batch_size = 8,
                    verbose=1,
                    epochs=150,
                    validation_data=(X_val, y_val),
                    callbacks=callbacks,shuffle=False)

model.save('Fundus_test.hdf5')


In [ ]:
from sklearn.metrics import mean_absolute_error
from skimage.metrics import structural_similarity, peak_signal_noise_ratio
from sklearn.metrics import accuracy_score,mean_squared_error, recall_score, precision_score, f1_score
import numpy as np
from sklearn.metrics import accuracy_score
def compute_iou(im1, im2):
    overlap = (im1>0.5) * (im2>0.5)
    union = (im1>0.5) + (im2>0.5)
    return overlap.sum()/float(union.sum())


# Calculate loss
def calculate_loss(y_true, y_pred):
    return mean_squared_error(y_true, y_pred)

# Evaluation metric: Dice
def compute_dice(im1, im2, empty_score=1.0):
    im1 = np.asarray(im1>0.5).astype(np.bool)
    im2 = np.asarray(im2>0.5).astype(np.bool)

    if im1.shape != im2.shape:
        raise ValueError("Shape mismatch: im1 and im2 must have the same shape.")

    im_sum = im1.sum() + im2.sum()
    if im_sum == 0:
        return empty_score

    intersection = np.logical_and(im1, im2)

    return 2. * intersection.sum() / im_sum

#model loading
model.load_weights('model path')

for i in range(1,50):
    test_img = X_test[i]
    ground_truth=y_test[i]
    test_img=np.array(test_img, dtype=float)
    test_img = np.expand_dims(test_img, axis=0)

    prediction = (model.predict(test_img) > 0.2)
    ground_truth_arg=np.argmax(ground_truth,axis=1)
    prediction_arg = np.argmax(prediction,axis=1)
    #prediction=np.argmax(prediction, axis=1)
    #print(prediction_arg.shape)
    #print(ground_truth_arg.shape)
    #iou=compute_iou(prediction[0,:,:,0],ground_truth[:,:])
    #print(iou)
    #dice=compute_dice(prediction[0,:,:,0],ground_truth[:,:])
    #print(dice)
    #print(ground_truth.shape)
    #print(prediction.shape)

    #mae=mean_absolute_error(ground_truth[:,:], prediction[0,:,:,0])
    #print(mae)
    #ssim=structural_similarity(ground_truth[:,:], prediction[0,:,:,0])
    #print(ssim)
    #psnr=peak_signal_noise_ratio(ground_truth[:,:], prediction[0,:,:,0],data_range=1)
    #print(psnr)
    mse = mean_squared_error(ground_truth[:,:], prediction[0,:,:,0])
    print(mse)
    #print("Loss:", loss)



